# Gradient Boosting Classifier

## Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 모델의 성능 평가 지표를 뽑아주는 함수 정의
def evaluate_model(y_true, y_pred):
    conf_mat = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    # classify 문제에 softmax 방식을 적용하기 위해 multi_class 인수 추가
#     auc = roc_auc_score(y_true, y_pred)#, multi_class='multinomial') 
    fpr, tpr, thresholds = roc_curve(y_true,y_pred)
    print("Confusion Matrix:\n", conf_mat)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
#     print("AUC:", auc)

In [4]:
def eval_model(y_true, y_pred):
    # Confusion Matrix
    confusion_mat = confusion_matrix(y_true, y_pred, labels=[4, 5, 6, 7, 8])
    print(confusion_mat)
    
    # Total Accuracy
    accuracy = accuracy_score(y_true, y_pred) # 전체 정확도
    print(accuracy)
    
    # Class Accuracy & Macro / Weighted Accuracy
    class_names = [4, 5, 6, 7, 8] # 클래스 이름
    macro_accuracy = 0
    weighted_accuracy = 0

    for class_name in class_names:
        indices = [i for i, x in enumerate(y_true) if x == class_name] # 해당 클래스의 샘플 인덱스
        y_true_class = [y_true[i] for i in indices]
        y_pred_class = [y_pred[i] for i in indices]
        accuracy_class = accuracy_score(y_true_class, y_pred_class) # 해당 클래스의 정확도
        precision_class = precision_score(y_true, y_pred, labels=[class_name], average='micro')
        recall_class = recall_score(y_true, y_pred, labels=[class_name], average='micro')
        f1_class = f1_score(y_true, y_pred, labels=[class_name], average='micro')
        print("Class", class_name)
        print(class_name, "class Accuracy: ", accuracy_class)
        print(class_name, "class Precision: ", precision_class)
        print(class_name, "class Recall: ", recall_class)
        print(class_name, "class f1: ", f1_class)
        macro_accuracy += accuracy_class
        weighted_accuracy += accuracy_class * len(indices)

    macro_accuracy /= len(class_names) # 매크로 평균
    weighted_accuracy /= len(y_true) # 가중 평균
    print("Macro-Average Accuracy:", macro_accuracy)
    print("Weighted-Average Accuracy:", weighted_accuracy)

## Data 기본 처리
결측치 행 삭제, type column의 string 처리, train-validation-test set 분리

In [5]:
df = pd.read_csv("winequality.csv")
df

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [6]:
# 결측치 행 삭제
df = df.dropna() 

# string인 type column을 0과 1로 변경
df['type'] = (df['type']=='white').astype(int)

# observation이 적은 quality 3, 9인 케이스를 각각 4, 8 케이스에 합치기
df['quality'] = df['quality'].replace({3:4, 9:8})

In [7]:
# 데이터셋을 트레이닝 셋과 임시 셋으로 60:40 비율로 나눔
X_train, X_tmp, y_train, y_tmp = train_test_split(df.iloc[:, :11], df.iloc[:, 12:], test_size=0.4, random_state=100)
# oversampling(ros), undersampling(rus)
ros = RandomOverSampler(random_state=100)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)
rus = RandomUnderSampler(random_state=100)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)
# 임시 셋을 밸리데이션 셋과 테스트 셋으로 50:50 비율로 나눔
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=100)

##### Train - Validation - Test : 60 - 20 - 20으로 나눔
X_train, X_tmp, y_train, y_tmp = train_test_split(df.iloc[:, :11], df.iloc[:, 12:], test_size=0.4, random_state=100)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=100)

## Model GridSearch

multi_class 변수(class의 수 적으면 ovr, 반대는 multinomial을 주로 쓴다)를 Grid Search를 통해 알 수 없으므로, 두 번의 그리드서치를 통해 더 나은 것을 선택한다. <br>
-> 그 결과 ovr보다 multinomial이 작지만 약간 더 좋았다. <br>
ros와 rus에 대해 각각 gridsearch를 진행해본다. 

처음 trial 결과. 이후 over/undersampling 적용 과정에서 이 결과를 바탕으로 gridsearch 재설정
GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=100),
             param_grid=[{'learning_rate': [0.01, 0.1, 1],
                          'max_depth': [3, 5, 7], 'min_samples_leaf': [1, 2, 4],
                          'min_samples_split': [2, 5, 10],
                          'n_estimators': [50, 100, 200],
                          'subsample': [0.5, 0.7, 1]}],
             return_train_score=True)
{'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.7}

In [8]:
# Gradient Boosting Classifier Model 선언
gb_model = GradientBoostingClassifier(random_state=100)

# hyperparameter 설정
param_grid = [{
    # n_estimators: 사용할 트리의 개수. 시간과 비례
    'n_estimators': [75, 100, 150],
    # max_depth: 트리의 최대 깊이. 클수록 overfitting 위험.
    'max_depth': [7, 9, 11],
    # learning_rate: 숫자가 작을수록 세밀한 학습이 가능하나 시간이 많이 걸림. 
    'learning_rate': [0.05, 0.1, 0.5],
    # subsample: 각 트리에서 사용할 전체 데이터 대비 sample 데이터의 양. overfitting을 막기 위함.
    'subsample': [0.6, 0.7, 0.8],
    # min_sample_split: 노드를 분할하기 위한 최소한의 sample 수. overfitting을 막기 위함.
    'min_samples_split': [2],
    # min_sample_leaf: leaf node가 되기 위한 최소한의 sample 수. overfitting을 막기 위함.
    'min_samples_leaf': [4, 8]
}]

In [9]:
# Grid Search로 최적의 hyperparameter search
ros_gb_grid_search = GridSearchCV(gb_model, param_grid, n_jobs=-1, verbose=2, scoring='accuracy')
ros_gb_grid_search.fit(X_train_ros, y_train_ros)

# 최적 하이퍼파라미터 및 Accuracy score 출력
print("<<<ros grid search result>>>")
print('Best parameters:', ros_gb_grid_search.best_params_)
print('Best Accuracy score:', ros_gb_grid_search.best_score_)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

...

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<<<ros grid search result>>>
Best parameters: {'learning_rate': 0.1, 'max_depth': 11, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 150, 'subsample': 0.8}
Best Accuracy score: 0.8964454976303318


#z Grid Search 결과 출력
print(gb_grid_search.cv_results_)

In [11]:
# Grid Search로 최적의 hyperparameter search
rus_gb_grid_search = GridSearchCV(gb_model, param_grid, n_jobs=-1, verbose=2, scoring='accuracy')
rus_gb_grid_search.fit(X_train_rus, y_train_rus)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

...

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

GridSearchCV(estimator=GradientBoostingClassifier(random_state=100), n_jobs=-1,
             param_grid=[{'learning_rate': [0.05, 0.1, 0.5],
                          'max_depth': [7, 9, 11], 'min_samples_leaf': [4, 8],
                          'min_samples_split': [2],
                          'n_estimators': [75, 100, 150],
                          'subsample': [0.6, 0.7, 0.8]}],
             scoring='accuracy', verbose=2)

In [12]:
# Grid Search로 찾은 hyperparameter 출력
# print(gb_grid_search.best_params_)

# 최적 하이퍼파라미터 및 F1 score 출력
print("<<<rus grid search result>>>")
print('Best parameters:', rus_gb_grid_search.best_params_)
print('Best Accuracy score:', rus_gb_grid_search.best_score_)

<<<rus grid search result>>>
Best parameters: {'learning_rate': 0.05, 'max_depth': 11, 'min_samples_leaf': 8, 'min_samples_split': 2, 'n_estimators': 75, 'subsample': 0.8}
Best Accuracy score: 0.47619047619047616
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.6; total time=   0.7s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.7; total time=   0.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.6; total time=   1.0s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.7; total time=   1.1s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.6; total time=   1.5s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.8

[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.6; total time=   0.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.8; total time=   0.9s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.7; total time=   1.1s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   1.2s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.7; total time=   1.6s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=8, min_samples_split=2, n_estimators=75, subsample=0.8; total time=   0.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf

[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.6; total time=   0.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.7; total time=   0.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.6; total time=   1.0s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   1.2s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.6; total time=   1.5s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=8, min_samples_split=2, n_estimators=75, subsample=0.7; total time=   0.7s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf

[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.7; total time=   0.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.8; total time=   0.9s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.7; total time=   1.1s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   1.2s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.7; total time=   1.7s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=8, min_samples_split=2, n_estimators=75, subsample=0.6; total time=   0.6s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=8, min_samples_split=2, n_estimators=75, subsample=0.6; total time=   0.6s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=

[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.6; total time=   0.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=75, subsample=0.8; total time=   0.9s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.6; total time=   1.0s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   1.2s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.6; total time=   1.5s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=8, min_samples_split=2, n_estimators=75, subsample=0.7; total time=   0.7s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf

In [14]:
# 선택한 hyperparameter로 ros 모델 학습 - multinomial: softmax 형식으로
gb_model_ros = GradientBoostingClassifier(n_estimators = 150,
                                                 max_depth = 11,
                                                 learning_rate = 0.1,
                                                 subsample = 0.8,
                                                 min_samples_split = 2,
                                                 min_samples_leaf = 4,
                                                 loss = 'deviance')
gb_model_ros.multi_class = 'multinomial'
gb_model_ros.fit(X_train_ros, y_train_ros)
gb_pred_ros = pd.DataFrame(gb_model_ros.predict(X_val)).rename(columns={0:'pred_ros'})
y_val = y_val.reset_index()
gb_pred_ros = pd.concat([gb_pred_ros, y_val.quality], axis = 1)
print("<<<ros validation result>>>")
eval_model(gb_pred_ros.quality, gb_pred_ros.pred_ros)

<<<ros validation result>>>
[[  6  21  11   0   0]
 [  8 295 127   3   1]
 [  3 107 432  33   1]
 [  0   4  92 113   0]
 [  0   0  16  11   9]]
0.6612529002320185
Class 4
4 class Accuracy:  0.15789473684210525
4 class Precision:  0.35294117647058826
4 class Recall:  0.15789473684210525
4 class f1:  0.21818181818181817
Class 5
5 class Accuracy:  0.6797235023041475
5 class Precision:  0.6908665105386417
5 class Recall:  0.6797235023041475
5 class f1:  0.6852497096399536
Class 6
6 class Accuracy:  0.75
6 class Precision:  0.6371681415929203
6 class Recall:  0.75
6 class f1:  0.6889952153110047
Class 7
7 class Accuracy:  0.5406698564593302
7 class Precision:  0.70625
7 class Recall:  0.5406698564593302
7 class f1:  0.6124661246612466
Class 8
8 class Accuracy:  0.25
8 class Precision:  0.8181818181818182
8 class Recall:  0.25
8 class f1:  0.38297872340425526
Macro-Average Accuracy: 0.4756576191211166
Weighted-Average Accuracy: 0.6612529002320185


In [35]:
gb_model_ros.feature_importances_
X_train_ros.columns

Index(['type', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates'],
      dtype='object')

In [15]:
# 선택한 hyperparameter로 rus 모델 학습 - multinomial: softmax 형식으로
gb_model_rus = GradientBoostingClassifier(n_estimators = 75,
                                                 max_depth = 11,
                                                 learning_rate = 0.05,
                                                 subsample = 0.8,
                                                 min_samples_split = 2,
                                                 min_samples_leaf = 8,
                                                 loss = 'deviance')
gb_model_rus.multi_class = 'multinomial'
gb_model_rus.fit(X_train_rus, y_train_rus)
gb_pred_rus = pd.DataFrame(gb_model_rus.predict(X_val)).rename(columns={0:'pred_rus'})
# y_val = y_val.reset_index()
gb_pred_rus = pd.concat([gb_pred_rus, y_val.quality], axis = 1)
print("<<<rus validation result>>>")
eval_model(gb_pred_rus.quality, gb_pred_rus.pred_rus)

<<<rus validation result>>>
[[ 26   7   3   2   0]
 [ 98 192  96  34  14]
 [ 62 113 184 135  82]
 [ 11  13  33  85  67]
 [  1   1   3   7  24]]
0.3952049497293117
Class 4
4 class Accuracy:  0.6842105263157895
4 class Precision:  0.13131313131313133
4 class Recall:  0.6842105263157895
4 class f1:  0.22033898305084748
Class 5
5 class Accuracy:  0.4423963133640553
5 class Precision:  0.588957055214724
5 class Recall:  0.4423963133640553
5 class f1:  0.505263157894737
Class 6
6 class Accuracy:  0.3194444444444444
6 class Precision:  0.5768025078369906
6 class Recall:  0.3194444444444444
6 class f1:  0.41117318435754185
Class 7
7 class Accuracy:  0.40669856459330145
7 class Precision:  0.3231939163498099
7 class Recall:  0.40669856459330145
7 class f1:  0.36016949152542377
Class 8
8 class Accuracy:  0.6666666666666666
8 class Precision:  0.12834224598930483
8 class Recall:  0.6666666666666666
8 class f1:  0.2152466367713005
Macro-Average Accuracy: 0.5038833030768515
Weighted-Average Accurac

___

두번째 결과도 만족스럽지 않아 다시 grid search를 하고자 한다. <br>
1. ros

In [16]:
# Gradient Boosting Classifier Model 선언
gb_model_ros = GradientBoostingClassifier(random_state=100)

# hyperparameter 설정
param_grid_ros = [{
    # n_estimators: 사용할 트리의 개수. 시간과 비례
    'n_estimators': [150, 200, 300],
    # max_depth: 트리의 최대 깊이. 클수록 overfitting 위험.
    'max_depth': [11, 13, 15],
    # learning_rate: 숫자가 작을수록 세밀한 학습이 가능하나 시간이 많이 걸림. 
    'learning_rate': [0.1],
    # subsample: 각 트리에서 사용할 전체 데이터 대비 sample 데이터의 양. overfitting을 막기 위함.
    'subsample': [0.8, 0.9],
    # min_sample_split: 노드를 분할하기 위한 최소한의 sample 수. overfitting을 막기 위함.
    'min_samples_split': [2],
    # min_sample_leaf: leaf node가 되기 위한 최소한의 sample 수. overfitting을 막기 위함.
    'min_samples_leaf': [4]
}]

# Grid Search로 최적의 hyperparameter search
ros_gb_grid_search = GridSearchCV(gb_model, param_grid_ros, n_jobs=-1, verbose=2, scoring='accuracy')
ros_gb_grid_search.fit(X_train_ros, y_train_ros)

# 최적 하이퍼파라미터 및 Accuracy score 출력
print("<<<ros grid search result>>>")
print('Best parameters:', ros_gb_grid_search.best_params_)
print('Best Accuracy score:', ros_gb_grid_search.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/admin/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

<<<ros grid search result>>>
Best parameters: {'learning_rate': 0.1, 'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200, 'subsample': 0.8}
Best Accuracy score: 0.8994075829383888
[CV] END learning_rate=0.1, max_depth=11, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.8; total time=  20.8s
[CV] END learning_rate=0.1, max_depth=11, min_samples_leaf=4, min_samples_split=2, n_estimators=200, subsample=0.8; total time=  27.5s
[CV] END learning_rate=0.1, max_depth=11, min_samples_leaf=4, min_samples_split=2, n_estimators=200, subsample=0.9; total time=  30.9s
[CV] END learning_rate=0.1, max_depth=11, min_samples_leaf=4, min_samples_split=2, n_estimators=300, subsample=0.9; total time=  42.3s
[CV] END learning_rate=0.1, max_depth=13, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.9; total time=  28.6s
[CV] END learning_rate=0.1, max_depth=13, min_samples_leaf=4, min_samples_split=2, n_estimators=200, subsample=0.

[CV] END learning_rate=0.1, max_depth=11, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.8; total time=  20.6s
[CV] END learning_rate=0.1, max_depth=11, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.9; total time=  22.9s
[CV] END learning_rate=0.1, max_depth=11, min_samples_leaf=4, min_samples_split=2, n_estimators=200, subsample=0.9; total time=  30.9s
[CV] END learning_rate=0.1, max_depth=11, min_samples_leaf=4, min_samples_split=2, n_estimators=300, subsample=0.8; total time=  40.1s
[CV] END learning_rate=0.1, max_depth=13, min_samples_leaf=4, min_samples_split=2, n_estimators=150, subsample=0.8; total time=  26.2s
[CV] END learning_rate=0.1, max_depth=13, min_samples_leaf=4, min_samples_split=2, n_estimators=200, subsample=0.8; total time=  35.0s
[CV] END learning_rate=0.1, max_depth=13, min_samples_leaf=4, min_samples_split=2, n_estimators=200, subsample=0.9; total time=  41.7s
[CV] END learning_rate=0.1, max_depth=13, min_samples_l

In [18]:
# 선택한 hyperparameter로 ros 모델 학습 - multinomial: softmax 형식으로
gb_model_ros = GradientBoostingClassifier(n_estimators = 200,
                                                 max_depth = 15,
                                                 learning_rate = 0.1,
                                                 subsample = 0.8,
                                                 min_samples_split = 2,
                                                 min_samples_leaf = 4,
                                                 loss = 'deviance')
gb_model_ros.multi_class = 'multinomial'
gb_model_ros.fit(X_train_ros, y_train_ros)
gb_pred_ros = pd.DataFrame(gb_model_ros.predict(X_val)).rename(columns={0:'pred_ros'})
# y_val = y_val.reset_index()
gb_pred_ros = pd.concat([gb_pred_ros, y_val.quality], axis = 1)
print("<<<ros validation result>>>")
eval_model(gb_pred_ros.quality, gb_pred_ros.pred_ros)

<<<ros validation result>>>
[[  6  21  11   0   0]
 [  4 291 135   3   1]
 [  4 103 440  27   2]
 [  0   3 103 103   0]
 [  0   0  16  11   9]]
0.6566125290023201
Class 4
4 class Accuracy:  0.15789473684210525
4 class Precision:  0.42857142857142855
4 class Recall:  0.15789473684210525
4 class f1:  0.23076923076923078
Class 5
5 class Accuracy:  0.6705069124423964
5 class Precision:  0.6961722488038278
5 class Recall:  0.6705069124423964
5 class f1:  0.6830985915492958
Class 6
6 class Accuracy:  0.7638888888888888
6 class Precision:  0.624113475177305
6 class Recall:  0.7638888888888888
6 class f1:  0.6869633099141296
Class 7
7 class Accuracy:  0.49282296650717705
7 class Precision:  0.7152777777777778
7 class Recall:  0.49282296650717705
7 class f1:  0.5835694050991501
Class 8
8 class Accuracy:  0.25
8 class Precision:  0.75
8 class Recall:  0.25
8 class f1:  0.375
Macro-Average Accuracy: 0.4670227009361135
Weighted-Average Accuracy: 0.6566125290023201


---

미미하게 multinomial option이 더 좋게 나온 것을 확인할 수 있었다. <br>
이제 다른 옵션을 다시 변경해보며 더 좋은 예측력을 가질 수 있을지 확인 (다른 모델 구축 완료 후 시간 나면!)